In [ ]:
# Simple Single-Stage YOLOv9 Training for License Plate Detection
# Run this in Google Colab

# Cell 1: Install dependencies and setup
!pip install ultralytics kaggle opencv-python matplotlib
!pip install roboflow

import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from google.colab import files, drive
import shutil
from pathlib import Path
import yaml
import pandas as pd

print("All dependencies installed successfully!")

# Cell 2: Mount Drive and setup paths
drive.mount('/content/drive')

local_dataset_path = '/content/Bangla_License_Plate_Dataset'
yolo_dataset_path = '/content/yolo_dataset'
drive_backup_path = '/content/drive/MyDrive/Bangla_License_Plate_Dataset'

print("Drive mounted successfully!")

# Cell 3: Download and prepare dataset (same as your original)
dataset_ready = False

if os.path.exists(local_dataset_path):
    print("Dataset already exists locally")
    dataset_ready = True
elif os.path.exists(drive_backup_path):
    print("Copying dataset from Drive to local storage...")
    shutil.copytree(drive_backup_path, local_dataset_path)
    print("Dataset copied to local storage")
    dataset_ready = True
else:
    print("Dataset not found. Please upload your kaggle.json file:")
    uploaded = files.upload()

    os.makedirs('/root/.kaggle', exist_ok=True)
    shutil.move('kaggle.json', '/root/.kaggle/kaggle.json')
    os.chmod('/root/.kaggle/kaggle.json', 0o600)

    print("Downloading dataset from Kaggle...")
    os.system('kaggle datasets download -d nishat99/bangla-license-plate-detection -p /content')
    os.system('unzip -q /content/bangla-license-plate-detection.zip -d /content')

    if os.path.exists('/content/Bangla License Plate Dataset'):
        shutil.move('/content/Bangla License Plate Dataset', local_dataset_path)
        dataset_ready = True

        try:
            shutil.copytree(local_dataset_path, drive_backup_path)
            print("Dataset backed up to Drive")
        except:
            print("Drive backup failed, continuing with local dataset")

if not dataset_ready:
    print("ERROR: Dataset preparation failed!")
else:
    print(f"Dataset ready at: {local_dataset_path}")

# Cell 4: Convert annotations to YOLO format (same as original)
def mask_to_bbox(mask):
    """Convert binary mask to bounding box coordinates"""
    coords = np.where(mask > 127)
    if len(coords[0]) == 0:
        return []
    y_min, y_max = coords[0].min(), coords[0].max()
    x_min, x_max = coords[1].min(), coords[1].max()
    return [(x_min, y_min, x_max, y_max)]

def bbox_to_yolo_format(bbox, img_width, img_height):
    """Convert bounding box to YOLO format (normalized)"""
    x_min, y_min, x_max, y_max = bbox
    x_center = (x_min + x_max) / 2.0
    y_center = (y_min + y_max) / 2.0
    width = x_max - x_min
    height = y_max - y_min

    x_center_norm = x_center / img_width
    y_center_norm = y_center / img_height
    width_norm = width / img_width
    height_norm = height / img_height

    return f"0 {x_center_norm:.6f} {y_center_norm:.6f} {width_norm:.6f} {height_norm:.6f}"

def process_dataset_split(split_name, local_dataset_path, yolo_dataset_path):
    """Process one split with progress tracking"""
    img_folder = os.path.join(local_dataset_path, split_name, 'img')
    mask_folder = os.path.join(local_dataset_path, split_name, 'masks')

    print(f"\nProcessing {split_name}:")

    if not os.path.exists(img_folder) or not os.path.exists(mask_folder):
        print(f"ERROR: Missing folders for {split_name}")
        return 0

    yolo_img_dir = os.path.join(yolo_dataset_path, split_name, 'images')
    yolo_label_dir = os.path.join(yolo_dataset_path, split_name, 'labels')
    os.makedirs(yolo_img_dir, exist_ok=True)
    os.makedirs(yolo_label_dir, exist_ok=True)

    img_files = [f for f in os.listdir(img_folder) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
    print(f"Found {len(img_files)} images in {split_name}")

    processed_count = 0

    for i, img_file in enumerate(img_files):
        if i % 500 == 0 and i > 0:
            print(f"  Processed {i}/{len(img_files)} images ({i/len(img_files)*100:.1f}%)")

        try:
            mask_name = os.path.splitext(img_file)[0] + '.png'
            mask_path = os.path.join(mask_folder, mask_name)

            if not os.path.exists(mask_path):
                continue

            mask = cv2.imread(mask_path, 0)
            if mask is None:
                continue

            img_path = os.path.join(img_folder, img_file)
            img = cv2.imread(img_path)
            if img is None:
                continue

            img_height, img_width = img.shape[:2]
            bboxes = mask_to_bbox(mask)

            # Copy image
            dst_img_path = os.path.join(yolo_img_dir, img_file)
            shutil.copy2(img_path, dst_img_path)

            # Create label file
            label_file = os.path.splitext(img_file)[0] + '.txt'
            label_path = os.path.join(yolo_label_dir, label_file)

            with open(label_path, 'w') as f:
                for bbox in bboxes:
                    yolo_line = bbox_to_yolo_format(bbox, img_width, img_height)
                    f.write(yolo_line + '\n')

            processed_count += 1

        except Exception as e:
            print(f"Error processing {img_file}: {e}")
            continue

    print(f"{split_name.upper()} processed: {processed_count} images")
    return processed_count

# Check if annotations already converted
annotations_complete_flag = os.path.join(yolo_dataset_path, 'annotations_complete.txt')

if os.path.exists(annotations_complete_flag):
    print("Annotations already exist! Skipping annotation generation...")
else:
    print("Creating YOLO annotations from masks...")

    train_count = process_dataset_split('train', local_dataset_path, yolo_dataset_path)
    val_count = process_dataset_split('validation', local_dataset_path, yolo_dataset_path)
    test_count = process_dataset_split('test', local_dataset_path, yolo_dataset_path)

    total_count = train_count + val_count + test_count
    print(f"\nANNOTATION CONVERSION COMPLETED!")
    print(f"Train: {train_count}, Validation: {val_count}, Test: {test_count}")
    print(f"Total: {total_count} images")

    with open(annotations_complete_flag, 'w') as f:
        f.write(f"Total: {total_count} images")

# Cell 5: Create dataset configuration
dataset_config = {
    'path': yolo_dataset_path,
    'train': 'train/images',
    'val': 'validation/images',
    'test': 'test/images',
    'nc': 1,
    'names': ['license_plate']
}

config_path = os.path.join(yolo_dataset_path, 'dataset.yaml')
with open(config_path, 'w') as f:
    yaml.dump(dataset_config, f)

print("Dataset configuration created!")



In [ ]:
import zipfile
import os
import shutil

# Path to your uploaded zip file
zip_path = '/content/yolomodel.zip'
extract_path = '/content/yolov7_models_restored'

# Check if zip file exists
if os.path.exists(zip_path):
    print(f"✓ Found zip file: {zip_path}")
    print(f"Size: {os.path.getsize(zip_path) / (1024*1024):.1f} MB")

    # Extract the zip file
    print(f"\nExtracting to: {extract_path}")
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_path)

    print("✓ Extraction complete!")

    # Show extracted contents
    print("\nExtracted contents:")
    for root, dirs, files in os.walk(extract_path):
        level = root.replace(extract_path, '').count(os.sep)
        indent = ' ' * 2 * level
        print(f"{indent}{os.path.basename(root)}/")
        subindent = ' ' * 2 * (level + 1)
        for file in files[:10]:  # Show first 10 files
            print(f"{subindent}{file}")
        if len(files) > 10:
            print(f"{subindent}... and {len(files) - 10} more files")

    # Find checkpoint files
    print("\n" + "="*60)
    print("Looking for checkpoint files...")
    print("="*60)

    checkpoint_files = []
    for root, dirs, files in os.walk(extract_path):
        for file in files:
            if file.endswith('.pt'):
                full_path = os.path.join(root, file)
                size_mb = os.path.getsize(full_path) / (1024*1024)
                checkpoint_files.append((full_path, size_mb))
                print(f"✓ Found: {file} ({size_mb:.1f} MB)")
                print(f"  Path: {full_path}")

    if checkpoint_files:
        # Find best.pt or last.pt
        best_pt = [f for f in checkpoint_files if 'best.pt' in f[0]]
        last_pt = [f for f in checkpoint_files if 'last.pt' in f[0]]

        print("\n" + "="*60)
        print("CHECKPOINT SUMMARY")
        print("="*60)

        if best_pt:
            print(f"✓ best.pt found at: {best_pt[0][0]}")
        if last_pt:
            print(f"✓ last.pt found at: {last_pt[0][0]}")

        # Copy to expected location for resume
        target_dir = '/kaggle/working/yolov7_models/yolov7_license_plate/weights'
        os.makedirs(target_dir, exist_ok=True)

        if last_pt:
            source = last_pt[0][0]
            dest = os.path.join(target_dir, 'last.pt')
            shutil.copy2(source, dest)
            print(f"\n✓ Copied last.pt to: {dest}")
            print("You can now resume training from this checkpoint!")

        if best_pt:
            source = best_pt[0][0]
            dest = os.path.join(target_dir, 'best.pt')
            shutil.copy2(source, dest)
            print(f"✓ Copied best.pt to: {dest}")
    else:
        print("✗ No .pt checkpoint files found in the zip")

else:
    print(f"✗ Zip file not found at: {zip_path}")
    print("Please upload the file first using:")
    print("  1. Click 'Add Data' button on the right panel")
    print("  2. Or use: from google.colab import files; files.upload()")

In [ ]:
#Cell 6: Single-Stage Training with Checkpoint Resume
from ultralytics import YOLO
import torch

# Setup paths
model_save_dir = '/content/yolov7_models_restored'
os.makedirs(model_save_dir, exist_ok=True)

print(f"GPU Available: {torch.cuda.is_available()}")

# Check GPU memory and recommend batch size
if torch.cuda.is_available():
    gpu_memory = torch.cuda.get_device_properties(0).total_memory / 1e9
    print(f"GPU Memory: {gpu_memory:.1f} GB")

    if gpu_memory < 8:
        recommended_batch = 4
        print("WARNING: Low GPU memory detected. Using batch size 4.")
    elif gpu_memory < 12:
        recommended_batch = 6
        print("Moderate GPU memory. Using batch size 6.")
    else:
        recommended_batch = 8
        print("High GPU memory. Using batch size 8.")
else:
    recommended_batch = 4
    print("No GPU detected. Using batch size 4.")

# Check for existing checkpoint to resume training
checkpoint_path = os.path.join(model_save_dir, 'single_stage', 'weights', 'last.pt')

if os.path.exists(checkpoint_path):
    print("\n" + "="*60)
    print("FOUND EXISTING CHECKPOINT - RESUMING TRAINING")
    print("="*60)
    model = YOLO(checkpoint_path)
    print(f"Loaded checkpoint from: {checkpoint_path}")
    resume_training = True
else:
    print("\n" + "="*60)
    print("STARTING NEW TRAINING")
    print("="*60)
    model = YOLO('yolov9m.pt')
    print("Loaded fresh YOLOv9m model")
    resume_training = False

print("\n" + "="*60)
print("SINGLE-STAGE TRAINING (70 epochs)")
print("Checkpoints saved every 10 epochs")
print("="*60)

# Training with checkpoint saving
results = model.train(
    data=config_path,
    epochs=70,
    batch=recommended_batch,
    name='single_stage',
    project=model_save_dir,
    save_period=10,      # Save checkpoint every 10 epochs
    resume=resume_training,  # Resume if checkpoint exists

    # Standard YOLO hyperparameters
    freeze=0,
    lr0=0.01,
    lrf=0.01,
    momentum=0.937,
    weight_decay=0.0005,
    warmup_epochs=3,
    warmup_momentum=0.8,
    warmup_bias_lr=0.1,
    box=7.5,
    cls=0.5,
    dfl=1.5,

    # Standard augmentations
    hsv_h=0.015,
    hsv_s=0.7,
    hsv_v=0.4,
    degrees=0.0,
    translate=0.1,
    scale=0.5,
    shear=0.0,
    perspective=0.0,
    flipud=0.0,
    fliplr=0.5,
    mosaic=1.0,
    mixup=0.0,
    copy_paste=0.0,

    patience=25,
    exist_ok=True,
    verbose=True
)

print("\nTraining completed!")
print(f"Checkpoints saved in: {model_save_dir}/single_stage/weights/")
print("- last.pt (latest checkpoint)")
print("- best.pt (best performing model)")
print("- epoch10.pt, epoch20.pt, etc. (periodic checkpoints)")

# Load the best trained model
best_model_path = os.path.join(model_save_dir, 'single_stage', 'weights', 'best.pt')
if os.path.exists(best_model_path):
    trained_model = YOLO(best_model_path)
    print("\nBest model loaded successfully!")
else:
    print("\nWarning: Best model not found!")

In [ ]:

# Cell 7: Evaluation (same as your original)
def load_ground_truth_boxes(label_path, img_width, img_height):
    """Load ground truth boxes from YOLO format label file"""
    boxes = []
    if os.path.exists(label_path):
        with open(label_path, 'r') as f:
            lines = f.readlines()

        for line in lines:
            line = line.strip()
            if line:
                parts = line.split()
                if len(parts) == 5:
                    try:
                        _, x_center, y_center, width, height = map(float, parts)

                        x_center_abs = x_center * img_width
                        y_center_abs = y_center * img_height
                        width_abs = width * img_width
                        height_abs = height * img_height

                        x1 = x_center_abs - width_abs / 2
                        y1 = y_center_abs - height_abs / 2
                        x2 = x_center_abs + width_abs / 2
                        y2 = y_center_abs + height_abs / 2

                        boxes.append([x1, y1, x2, y2])
                    except ValueError:
                        continue
    return boxes

def calculate_box_iou(box1, box2):
    """Calculate IoU between two bounding boxes"""
    x1_1, y1_1, x2_1, y2_1 = box1
    x1_2, y1_2, x2_2, y2_2 = box2

    x1_i = max(x1_1, x1_2)
    y1_i = max(y1_1, y1_2)
    x2_i = min(x2_1, x2_2)
    y2_i = min(y2_1, y2_2)

    if x2_i <= x1_i or y2_i <= y1_i:
        return 0.0

    intersection_area = (x2_i - x1_i) * (y2_i - y1_i)
    box1_area = (x2_1 - x1_1) * (y2_1 - y1_1)
    box2_area = (x2_2 - x1_2) * (y2_2 - y1_2)
    union_area = box1_area + box2_area - intersection_area

    if union_area == 0:
        return 0.0

    return intersection_area / union_area

def evaluate_model(model, split_name, conf_thresh=0.5):
    """Comprehensive evaluation with same metrics as multi-stage"""
    img_dir = f'{yolo_dataset_path}/{split_name}/images'
    label_dir = f'{yolo_dataset_path}/{split_name}/labels'

    img_files = [f for f in os.listdir(img_dir) if f.endswith(('.jpg', '.png'))]

    tp, fp, fn, tn = 0, 0, 0, 0
    iou_scores = []

    print(f"Evaluating {len(img_files)} images...")

    for i, img_file in enumerate(img_files):
        if i % 500 == 0 and i > 0:
            print(f"Progress: {i}/{len(img_files)}")

        img_path = f'{img_dir}/{img_file}'
        label_path = f'{label_dir}/{os.path.splitext(img_file)[0]}.txt'

        img = cv2.imread(img_path)
        if img is None:
            continue
        h, w = img.shape[:2]

        gt_boxes = load_ground_truth_boxes(label_path, w, h)
        has_gt = len(gt_boxes) > 0

        results = model(img_path, conf=conf_thresh, verbose=False)
        pred_boxes = []
        if results[0].boxes is not None:
            for box in results[0].boxes:
                x1, y1, x2, y2 = box.xyxy[0].cpu().numpy()
                pred_boxes.append([x1, y1, x2, y2])
        has_pred = len(pred_boxes) > 0

        if has_gt and has_pred:
            max_iou = 0
            for gt_box in gt_boxes:
                for pred_box in pred_boxes:
                    iou = calculate_box_iou(gt_box, pred_box)
                    max_iou = max(max_iou, iou)

            iou_scores.append(max_iou)
            if max_iou > 0.5:
                tp += 1
            else:
                fn += 1
        elif has_gt and not has_pred:
            fn += 1
            iou_scores.append(0)
        elif not has_gt and has_pred:
            fp += 1
            iou_scores.append(0)
        else:
            tn += 1
            iou_scores.append(1)

    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0
    f1 = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0
    accuracy = (tp + tn) / len(img_files)
    avg_iou = sum(iou_scores) / len(iou_scores)
    binary_detection = sum(1 for iou in iou_scores if iou > 0.7) / len(iou_scores)

    return {
        'precision': precision,
        'recall': recall,
        'f1_score': f1,
        'accuracy': accuracy,
        'avg_iou': avg_iou,
        'binary_detection': binary_detection,
        'iou_scores': iou_scores
    }

# Evaluate the model
print("Evaluating simple YOLO model...")
val_results = evaluate_model(trained_model, 'validation')
test_results = evaluate_model(trained_model, 'test')

print("\n" + "="*50)
print("SIMPLE SINGLE-STAGE YOLO RESULTS")
print("="*50)

print("\nVALIDATION RESULTS:")
print(f"IoU: {val_results['avg_iou']:.4f}")
print(f"Precision: {val_results['precision']:.4f}")
print(f"Recall: {val_results['recall']:.4f}")
print(f"F1-Score: {val_results['f1_score']:.4f}")
print(f"Binary Detection (IoU>0.7): {val_results['binary_detection']:.4f}")

print("\nTEST RESULTS:")
print(f"IoU: {test_results['avg_iou']:.4f}")
print(f"Precision: {test_results['precision']:.4f}")
print(f"Recall: {test_results['recall']:.4f}")
print(f"F1-Score: {test_results['f1_score']:.4f}")
print(f"Binary Detection (IoU>0.7): {test_results['binary_detection']:.4f}")
print(f"Boundary Box Accuracy: {test_results['binary_detection']*100:.1f}%")

# Cell 8: Training Visualization (same as multi-stage)
results_csv = f'{model_save_dir}/single_stage/results.csv'

if os.path.exists(results_csv):
    data = pd.read_csv(results_csv)

    fig, axes = plt.subplots(2, 2, figsize=(12, 8))
    epochs = range(1, len(data) + 1)

    # Loss
    axes[0,0].plot(epochs, data['train/box_loss'], 'b-', label='Train Box Loss')
    if 'val/box_loss' in data.columns:
        axes[0,0].plot(epochs, data['val/box_loss'], 'r-', label='Val Box Loss')
    axes[0,0].set_title('Box Loss')
    axes[0,0].legend()
    axes[0,0].grid(True)

    # mAP
    axes[0,1].plot(epochs, data['metrics/mAP50(B)'], 'g-', label='mAP@0.5')
    axes[0,1].set_title('mAP@0.5')
    axes[0,1].legend()
    axes[0,1].grid(True)

    # Precision & Recall
    axes[1,0].plot(epochs, data['metrics/precision(B)'], 'purple', label='Precision')
    axes[1,0].plot(epochs, data['metrics/recall(B)'], 'brown', label='Recall')
    axes[1,0].set_title('Precision & Recall')
    axes[1,0].legend()
    axes[1,0].grid(True)

    # Final metrics comparison
    metrics = ['IoU', 'Precision', 'Recall', 'F1', 'Binary Det']
    test_scores = [test_results['avg_iou'], test_results['precision'],
                   test_results['recall'], test_results['f1_score'],
                   test_results['binary_detection']]

    axes[1,1].bar(metrics, test_scores, alpha=0.7)
    axes[1,1].set_title('Final Test Performance')
    axes[1,1].set_xticklabels(metrics, rotation=45)
    axes[1,1].grid(True, alpha=0.3)

    plt.tight_layout()
    plt.show()

    print(f"\nTraining Summary:")
    print(f"Total epochs: {len(data)}")
    print(f"Best mAP@0.5: {data['metrics/mAP50(B)'].max():.4f}")
    print(f"Final Test IoU: {test_results['avg_iou']:.4f}")

print("\nSimple single-stage YOLO training completed!")
print("Now you can compare with your multi-stage results!")

In [ ]:
import shutil
from google.colab import files, drive



# Specify the folder path you want to zip
folder_path = '/content/yolov7_models_restored'
zip_name = 'yolomodel2'

# Create a zip file of the folder
shutil.make_archive(zip_name, 'zip', folder_path)

# Save to Google Drive
drive_path = '/content/drive/MyDrive/yolomodel2.zip'
shutil.move(f'{zip_name}.zip', drive_path)
print(f"Saved to Google Drive: {drive_path}")

# Also download to your local machine
#shutil.make_archive(zip_name, 'zip', folder_path)
#files.download(f'{zip_name}.zip')
print(f"Downloaded {zip_name}.zip to your computer successfully!")